In [28]:
import pandas as pd

In [29]:
base_vendas = pd.read_excel('bases/relatorio_notas.xlsx')
display(base_vendas)

,Numero nota fiscal,Data da venda,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor total (nota),Quantidade (volume nota),Valor desconto,Valor total produtos,Unidade
0,564555,06/06/2025,1093,CANETA ESF CR MEDIA 1.0 AZ BIC,100.0,0.620000,0.590950,10.0,0.35,1047.13,UN
1,564555,06/06/2025,3708,PAPEL A4 SENNINHA AZ 75G 100F REPORT,25.0,5.460000,5.132735,10.0,2.69,1047.13,PC
2,564555,06/06/2025,3711,PAPEL A4 SENNINHA VD 75G 100F REPORT,25.0,5.460000,5.133527,10.0,2.69,1047.13,PC
3,564555,06/06/2025,712,REAB MARC PERM AZ 37ML PILOT,12.0,8.120000,7.818223,10.0,4.55,1047.13,UN
4,564555,06/06/2025,12815,QUADRO HORARIO TRABALHO HORIZON DAC 046F,40.0,6.850000,6.195026,10.0,3.84,1047.13,UN
...,...,...,...,...,...,...,...,...,...,...,...
316347,403,03/01/2025,31327,CAIXA TERMICA PT 34L MOR,20.0,71.595537,71.595537,10.0,NaN,NaN,UN
316348,403,03/01/2025,31329,CAIXA TERMICA VM 18L MOR,52.0,44.910000,44.910000,10.0,NaN,NaN,UN
316349,403,03/01/2025,9552,CX ISOPOR 33L ISOPLAST,6.0,25.542042,25.998995,10.0,NaN,NaN,UN
316350,402,03/01/2025,6792,CX ISOPOR 3L ISOPLAST,24.0,2.780000,3.172271,1.0,NaN,NaN,UN


In [30]:
base_vendas.columns.tolist()

['Numero nota fiscal',
 'Data da venda',
 'Código produto',
 'Descrição do produto',
 'Quantidade do produto',
 'Valor unitário',
 'Valor total (nota)',
 'Quantidade (volume nota)',
 'Valor desconto',
 'Valor total produtos',
 'Unidade']

In [31]:
base_vendas.isnull().sum()

Numero nota fiscal             0
Data da venda                  0
Código produto                 0
Descrição do produto           0
Quantidade do produto          0
Valor unitário                 0
Valor total (nota)             0
Quantidade (volume nota)    1700
Valor desconto              2992
Valor total produtos        2710
Unidade                        0
dtype: int64

In [32]:
base_vendas.describe()

,Numero nota fiscal,Código produto,Quantidade do produto,Valor unitário,Valor total (nota),Quantidade (volume nota),Valor desconto,Valor total produtos
count,316352.000000,316352.000000,316352.000000,316352.000000,316352.000000,314652.000000,313360.000000,313642.000000
mean,479163.187396,22072.866016,15.675853,15.841829,13.089965,28.499574,5.588534,2089.542827
std,181381.251400,11739.467571,84.047683,28.998172,23.923867,29.011042,11.322789,3690.735347
min,401.000000,5.000000,-3125.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,544042.000000,11452.000000,2.000000,3.960000,3.184941,11.000000,1.100000,501.510000
50%,550455.000000,24782.000000,5.000000,8.180000,6.649376,19.000000,2.580000,1074.740000
75%,557628.000000,32222.000000,12.000000,18.700000,15.165985,36.000000,6.450000,2334.170000
max,564555.000000,38229.000000,9764.000000,1169.000000,974.762703,238.000000,569.680000,141784.350000


In [33]:
colunas_uteis = [
    'Numero nota fiscal', 
    'Código produto', 
    'Descrição do produto',
    'Quantidade do produto', 
    'Valor unitário', 
    'Valor total produtos',
    'Valor total (nota)'
]

df_vendas = base_vendas[colunas_uteis].copy()

In [34]:
valor_nota = df_vendas.pop('Valor total produtos')
df_vendas['valor_nota'] = valor_nota

In [35]:

df_vendas.rename(columns={'Valor total (nota)': 'Valor pago'}, inplace=True)
df_vendas

,Numero nota fiscal,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota
0,564555,1093,CANETA ESF CR MEDIA 1.0 AZ BIC,100.0,0.620000,0.590950,1047.13
1,564555,3708,PAPEL A4 SENNINHA AZ 75G 100F REPORT,25.0,5.460000,5.132735,1047.13
2,564555,3711,PAPEL A4 SENNINHA VD 75G 100F REPORT,25.0,5.460000,5.133527,1047.13
3,564555,712,REAB MARC PERM AZ 37ML PILOT,12.0,8.120000,7.818223,1047.13
4,564555,12815,QUADRO HORARIO TRABALHO HORIZON DAC 046F,40.0,6.850000,6.195026,1047.13
...,...,...,...,...,...,...,...
316347,403,31327,CAIXA TERMICA PT 34L MOR,20.0,71.595537,71.595537,NaN
316348,403,31329,CAIXA TERMICA VM 18L MOR,52.0,44.910000,44.910000,NaN
316349,403,9552,CX ISOPOR 33L ISOPLAST,6.0,25.542042,25.998995,NaN
316350,402,6792,CX ISOPOR 3L ISOPLAST,24.0,2.780000,3.172271,NaN


In [36]:
df_vendas.isnull().sum()

Numero nota fiscal          0
Código produto              0
Descrição do produto        0
Quantidade do produto       0
Valor unitário              0
Valor pago                  0
valor_nota               2710
dtype: int64

In [37]:
print("% dos Valores faltantes")
df_vendas.isnull().sum()/df_vendas['Código produto'].count() * 100

% dos Valores faltantes


Numero nota fiscal       0.000000
Código produto           0.000000
Descrição do produto     0.000000
Quantidade do produto    0.000000
Valor unitário           0.000000
Valor pago               0.000000
valor_nota               0.856641
dtype: float64

In [38]:
df_vendas.dropna(subset=['valor_nota'],axis=0)
df_vendas


,Numero nota fiscal,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota
0,564555,1093,CANETA ESF CR MEDIA 1.0 AZ BIC,100.0,0.620000,0.590950,1047.13
1,564555,3708,PAPEL A4 SENNINHA AZ 75G 100F REPORT,25.0,5.460000,5.132735,1047.13
2,564555,3711,PAPEL A4 SENNINHA VD 75G 100F REPORT,25.0,5.460000,5.133527,1047.13
3,564555,712,REAB MARC PERM AZ 37ML PILOT,12.0,8.120000,7.818223,1047.13
4,564555,12815,QUADRO HORARIO TRABALHO HORIZON DAC 046F,40.0,6.850000,6.195026,1047.13
...,...,...,...,...,...,...,...
316347,403,31327,CAIXA TERMICA PT 34L MOR,20.0,71.595537,71.595537,NaN
316348,403,31329,CAIXA TERMICA VM 18L MOR,52.0,44.910000,44.910000,NaN
316349,403,9552,CX ISOPOR 33L ISOPLAST,6.0,25.542042,25.998995,NaN
316350,402,6792,CX ISOPOR 3L ISOPLAST,24.0,2.780000,3.172271,NaN


In [39]:
df_vendas_limpo = df_vendas[df_vendas['valor_nota'].fillna(0) != 0].copy()
df_vendas_limpo

,Numero nota fiscal,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota
0,564555,1093,CANETA ESF CR MEDIA 1.0 AZ BIC,100.0,0.620000,0.590950,1047.13
1,564555,3708,PAPEL A4 SENNINHA AZ 75G 100F REPORT,25.0,5.460000,5.132735,1047.13
2,564555,3711,PAPEL A4 SENNINHA VD 75G 100F REPORT,25.0,5.460000,5.133527,1047.13
3,564555,712,REAB MARC PERM AZ 37ML PILOT,12.0,8.120000,7.818223,1047.13
4,564555,12815,QUADRO HORARIO TRABALHO HORIZON DAC 046F,40.0,6.850000,6.195026,1047.13
...,...,...,...,...,...,...,...
315799,16916,8802,AGENDA ANUAL CD COST NAPOLI 176F TILIBRA,4.0,35.770000,21.749064,87.00
315800,16915,21602,CANETA DETEC CEDULA FALSA BLT/1 JOCAR,7.0,4.285714,2.769312,19.39
315801,16914,21602,CANETA DETEC CEDULA FALSA BLT/1 JOCAR,7.0,4.285714,2.769312,19.39
315802,16913,36564,AGENDA ESP ANUAL 336F A5 HELLO DAC,1.0,37.620000,25.529850,25.53


In [40]:
df_vendas_limpo['Quantidade do produto'] = df_vendas_limpo['Quantidade do produto'].astype(int)
df_vendas_limpo['Quantidade do produto'] = df_vendas_limpo['Quantidade do produto'].astype(int)

In [41]:
colunas_floats = ['Valor unitário', 'Valor pago', 'valor_nota']
df_vendas_limpo[colunas_floats] = (df_vendas_limpo[colunas_floats].round(2).astype(float))
df_vendas_limpo

,Numero nota fiscal,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota
0,564555,1093,CANETA ESF CR MEDIA 1.0 AZ BIC,100,0.62,0.59,1047.13
1,564555,3708,PAPEL A4 SENNINHA AZ 75G 100F REPORT,25,5.46,5.13,1047.13
2,564555,3711,PAPEL A4 SENNINHA VD 75G 100F REPORT,25,5.46,5.13,1047.13
3,564555,712,REAB MARC PERM AZ 37ML PILOT,12,8.12,7.82,1047.13
4,564555,12815,QUADRO HORARIO TRABALHO HORIZON DAC 046F,40,6.85,6.20,1047.13
...,...,...,...,...,...,...,...
315799,16916,8802,AGENDA ANUAL CD COST NAPOLI 176F TILIBRA,4,35.77,21.75,87.00
315800,16915,21602,CANETA DETEC CEDULA FALSA BLT/1 JOCAR,7,4.29,2.77,19.39
315801,16914,21602,CANETA DETEC CEDULA FALSA BLT/1 JOCAR,7,4.29,2.77,19.39
315802,16913,36564,AGENDA ESP ANUAL 336F A5 HELLO DAC,1,37.62,25.53,25.53


In [42]:
df_vendas_limpo.nunique()

Numero nota fiscal       28293
Código produto            9053
Descrição do produto      8880
Quantidade do produto      411
Valor unitário            9364
Valor pago                8867
valor_nota               24727
dtype: int64

In [43]:
df_vendas_limpo.dtypes

Numero nota fiscal         int64
Código produto             int64
Descrição do produto      object
Quantidade do produto      int64
Valor unitário           float64
Valor pago               float64
valor_nota               float64
dtype: object

In [44]:
df_vendas_limpo.isnull().sum()

Numero nota fiscal       0
Código produto           0
Descrição do produto     0
Quantidade do produto    0
Valor unitário           0
Valor pago               0
valor_nota               0
dtype: int64

In [45]:
# Selecionar apenas as colunas numéricas
coluna_numericas = df_vendas_limpo.select_dtypes(include=['int', 'float']).columns
# contar os valores zerados em cada coluna 
zeros = (df_vendas_limpo[coluna_numericas] == 0).sum()
# Exibir os resultados
print("Valores zerados por colunas")
print(zeros[zeros > 0]) # mostra apenas colunas com valores zero

Valores zerados por colunas
Valor unitário    242
dtype: int64


In [53]:

df_vendas_limpo["Diferença real"] = df_vendas_limpo["Valor unitário"] - df_vendas_limpo["Valor pago"]
df_vendas_limpo["Deferença %"] = (df_vendas_limpo["Diferença real"] / df_vendas_limpo["Valor unitário"] * 100).round(2)
df_vendas_limpo


,Numero nota fiscal,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota,Diferença real,Deferença %
0,564555,1093,CANETA ESF CR MEDIA 1.0 AZ BIC,100,0.62,0.59,1047.13,0.03,4.84
1,564555,3708,PAPEL A4 SENNINHA AZ 75G 100F REPORT,25,5.46,5.13,1047.13,0.33,6.04
2,564555,3711,PAPEL A4 SENNINHA VD 75G 100F REPORT,25,5.46,5.13,1047.13,0.33,6.04
3,564555,712,REAB MARC PERM AZ 37ML PILOT,12,8.12,7.82,1047.13,0.30,3.69
4,564555,12815,QUADRO HORARIO TRABALHO HORIZON DAC 046F,40,6.85,6.20,1047.13,0.65,9.49
...,...,...,...,...,...,...,...,...,...
315799,16916,8802,AGENDA ANUAL CD COST NAPOLI 176F TILIBRA,4,35.77,21.75,87.00,14.02,39.19
315800,16915,21602,CANETA DETEC CEDULA FALSA BLT/1 JOCAR,7,4.29,2.77,19.39,1.52,35.43
315801,16914,21602,CANETA DETEC CEDULA FALSA BLT/1 JOCAR,7,4.29,2.77,19.39,1.52,35.43
315802,16913,36564,AGENDA ESP ANUAL 336F A5 HELLO DAC,1,37.62,25.53,25.53,12.09,32.14


In [52]:
df_vendas_limpo.describe()

c:\Users\Paulo Neto\Desktop\recomenda\venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Numero nota fiscal,Código produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota,Diferença real,Deferença %
count,312669.000000,312669.000000,312669.000000,312669.000000,312669.000000,312669.000000,312669.000000,312669.00
mean,480357.371284,22068.599836,15.375806,15.886220,13.114699,2096.045311,2.771521,-inf
std,180066.777280,11746.162487,76.256688,28.996984,23.886691,3694.629489,6.433829,NaN
min,16912.000000,5.000000,1.000000,0.000000,0.170000,1.040000,-200.640000,-inf
25%,544109.000000,11442.000000,2.000000,4.000000,3.210000,505.470000,0.600000,12.14
50%,550498.000000,24782.000000,5.000000,8.200000,6.690000,1080.630000,1.340000,17.22
75%,557628.000000,32245.000000,12.000000,18.750000,15.190000,2341.640000,2.980000,22.37
max,564555.000000,38229.000000,7680.000000,1169.000000,974.760000,141784.350000,373.970000,87.11


In [64]:
media_porcentagem = df_vendas_limpo["Deferença %"].mean()
desv_porcentagem = df_vendas_limpo["Deferença %"].std()
min_porcentagem = df_vendas_limpo["Deferença %"].min()
print(f"\nMédia : {media_porcentagem}\nDesvio Padrão : {desv_porcentagem}\nMínimo : {min_porcentagem}")


Média : -inf
Desvio Padrão : nan
Mínimo : -inf


c:\Users\Paulo Neto\Desktop\recomenda\venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [56]:
df_vendas_limpo[df_vendas_limpo["Deferença %"] == 87.11]

,Numero nota fiscal,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota,Diferença real,Deferença %
207948,546252,27702,MARC RETROP CD/DVD 2.0 AZ BRW CA5002,1,38.88,5.01,482.46,33.87,87.11


In [65]:
valor_mina = df_vendas_limpo[df_vendas_limpo["Valor pago"] == 0.170000]
valor_mina

,Numero nota fiscal,Código produto,Descrição do produto,Quantidade do produto,Valor unitário,Valor pago,valor_nota,Diferença real,Deferença %
223886,544968,7732,APONTADOR 1F RET LEO&LEONORA 4529,2016,0.21,0.17,16585.12,0.04,19.05
224060,544964,7732,APONTADOR 1F RET LEO&LEONORA 4529,576,0.21,0.17,4053.59,0.04,19.05
238616,543934,7732,APONTADOR 1F RET LEO&LEONORA 4529,24,0.20,0.17,7650.54,0.03,15.00
240549,543821,7732,APONTADOR 1F RET LEO&LEONORA 4529,144,0.21,0.17,13327.22,0.04,19.05
248596,543319,7732,APONTADOR 1F RET LEO&LEONORA 4529,24,0.21,0.17,2838.97,0.04,19.05
248642,543318,7732,APONTADOR 1F RET LEO&LEONORA 4529,24,0.21,0.17,4980.72,0.04,19.05
248690,543317,7732,APONTADOR 1F RET LEO&LEONORA 4529,24,0.21,0.17,4980.72,0.04,19.05
248805,543314,7732,APONTADOR 1F RET LEO&LEONORA 4529,24,0.21,0.17,5001.51,0.04,19.05
248809,543313,7732,APONTADOR 1F RET LEO&LEONORA 4529,24,0.21,0.17,5001.51,0.04,19.05
248877,543312,7732,APONTADOR 1F RET LEO&LEONORA 4529,48,0.21,0.17,4078.10,0.04,19.05
